In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
#import dataframe and drop nans
df_2021 = pd.read_csv('data/2021/2021_combined.csv')
#drop nans
df_2021 = df_2021.loc[df_2021.SK_CODE.isnull() != True]

In [3]:
# group trips by origin, destination and category
df_t = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming', 'SK_CODE']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :4].reset_index(drop=True)

In [4]:
# give columns english names
df_t.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

In [5]:
# subset inland trips
df_t = df_t.loc[(df_t.origin.str.contains('NL')) & (df_t.destination.str.contains('NL'))]
df_t.reset_index(inplace=True, drop=True)

In [6]:
#create dict to fill with total number of trips on path for each ship type
df_t_dict = {}
# now fill this dict
# loop over data frame
for i in range(len(df_t)):
    #define temp variables for ease
    origin = df_t['origin'][i]
    destination = df_t['destination'][i]
    ship_type = df_t['type'][i]
    trip_count = df_t['trips'][i]
    #only make new column if trip and its inverse are not in dict yet. Otherwise append.
    if (origin, destination, ship_type) in df_t_dict.keys():
        df_t_dict[(origin, destination, ship_type)] += trip_count
    elif (destination, origin, ship_type) in df_t_dict.keys():
        df_t_dict[(destination, origin, ship_type)] += trip_count
    else:
        df_t_dict[(origin, destination, ship_type)] = trip_count
df_t_dict

{('NLAMS', 'NLRTM', 'M8'): 3510,
 ('NLAMS', 'NLRTM', 'M12'): 2076,
 ('NLRTM', 'NLMOE', 'M8'): 1584,
 ('NLRTM', 'NLRTM', 'M8'): 715,
 ('NLAMS', 'NLRTM', 'M9'): 1316,
 ('NLRTM', 'NLWLK', 'M3'): 973,
 ('NLOOS', 'NLTLB', 'M8'): 688,
 ('NLRTM', 'NLAMS', 'M6'): 881,
 ('NLAER', 'NLAMS', 'BI'): 824,
 ('NLDHR', 'NLDHR', 'M0'): 411,
 ('NLTLB', 'NLRTM', 'M8'): 623,
 ('NLRTM', 'NLOOS', 'M8'): 727,
 ('NLHAR', 'NLWTE', 'M12'): 690,
 ('NLHGL', 'NLRTM', 'M8'): 652,
 ('NLRTM', 'NLVLI', 'M8'): 658,
 ('NLRTM', 'NLRTM', 'M12'): 363,
 ('NLMOE', 'NLOOS', 'M8'): 615,
 ('NLVEG', 'NLRTM', 'M6'): 575,
 ('NLAMS', 'NLRTM', 'M11'): 666,
 ('NLMOE', 'NLAPN', 'M8'): 604,
 ('NLVEG', 'NLRTM', 'M7'): 581,
 ('NLAMS', 'NLMOE', 'M8'): 588,
 ('NLRTM', 'NLMEP', 'M8'): 568,
 ('NLSTD', 'NLSTD', 'M0'): 282,
 ('NLAMS', 'NLLMR', 'BIIL-1'): 553,
 ('NLNIJ', 'NLWAS', 'M8'): 508,
 ('NLWSP', 'NLAER', 'M6'): 510,
 ('NLDOR', 'NLRTM', 'M8'): 459,
 ('NLAMS', 'NLHAS', 'M8'): 436,
 ('NLRTM', 'NLDZL', 'M8'): 415,
 ('NLRTM', 'NLTIE', 'M8'): 4

## Compute vessel sum per route
Idea: make similar dict as above first, but then with total amount of trips per route. Later on: add column for each ship.
This will result in a column for each ship type and an entry with the amount of ships of that type that have travelled.

In [7]:
# now only sort by origin and destination
all_trips_ranked = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :3].reset_index(drop=True)

#again, rename columns
all_trips_ranked.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

#remove non nl
all_trips_ranked = all_trips_ranked.loc[(all_trips_ranked.origin.str.contains('NL')) & (all_trips_ranked.destination.str.contains('NL'))]

#reset index to be able to loop over all values later on
all_trips_ranked.reset_index(inplace=True, drop=True)

# make dict to store trips
trip_dict_main = {}

In [8]:
# now fill this dict, same procedure as above
for i in range(len(all_trips_ranked)):
    origin = all_trips_ranked['origin'][i]
    destination = all_trips_ranked['destination'][i]
    trip_count = all_trips_ranked['trips'][i]

    if (origin, destination) in trip_dict_main.keys():
        trip_dict_main[(origin, destination)] += trip_count
    elif (destination, origin) in trip_dict_main.keys():
        trip_dict_main[(destination, origin)] += trip_count
    else:
        trip_dict_main[(origin, destination)] = trip_count

In [9]:
# now sort values
trip_dict_main = dict(sorted(trip_dict_main.items(), key=lambda item: item[1], reverse=True))

# make dataframe dict wih a key for each column
trip_dict_df_main = {'origin': [], 'destination': [], 'trip_count': []}
for i in df_2021.SK_CODE.unique():
    trip_dict_df_main[i] = []

# now fill this dict
for key, value in trip_dict_main.items():
    # append values from dict a
    trip_dict_df_main['origin'].append(key[0])
    trip_dict_df_main['destination'].append(key[1])
    trip_dict_df_main['trip_count'].append(value)
    #now append values from other dict, for each ship type
    for ship_type in df_2021.SK_CODE.unique():
        #if ships have travelled from A to B, append this value
        if (key[0], key[1], ship_type) in df_t_dict.keys():
            trip_dict_df_main[ship_type].append(df_t_dict[(key[0], key[1], ship_type)])
        else:
            try:
                # else, maybe trips on path A to B are stored as B to A
                trip_dict_df_main[ship_type].append(df_t_dict[(key[1], key[0], ship_type)])
            except:
                # no? Then no ships of this type have travelled this path, append 0
                trip_dict_df_main[ship_type].append(0)

In [10]:
df_main = pd.DataFrame.from_dict(trip_dict_df_main)

In [11]:
df_main

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2076,3510,1316,102,0,427,0,...,5,7,11,5,39,4,0,0,0,0
1,NLRTM,NLMOE,2686,70,1584,144,2,0,56,0,...,2,0,1,0,1,0,0,0,0,0
2,NLRTM,NLVLI,2153,190,658,414,10,0,140,0,...,0,2,2,1,0,0,0,1,0,0
3,NLAMS,NLAER,2119,6,132,0,2,0,7,0,...,0,1,5,37,0,3,0,1,1,0
4,NLRTM,NLRTM,1794,363,715,92,15,0,164,1,...,4,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10589,NLBZM,NLVER,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10590,NLBZM,NLSMB,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10591,NLBZM,NLSHH,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
#save dataframe
pickle.dump( df_main, open( "data/df_trips_per_path.p", "wb" ) )

## Generate random data

In [12]:
df_chance = df_main.copy()

In [13]:
df_chance

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2076,3510,1316,102,0,427,0,...,5,7,11,5,39,4,0,0,0,0
1,NLRTM,NLMOE,2686,70,1584,144,2,0,56,0,...,2,0,1,0,1,0,0,0,0,0
2,NLRTM,NLVLI,2153,190,658,414,10,0,140,0,...,0,2,2,1,0,0,0,1,0,0
3,NLAMS,NLAER,2119,6,132,0,2,0,7,0,...,0,1,5,37,0,3,0,1,1,0
4,NLRTM,NLRTM,1794,363,715,92,15,0,164,1,...,4,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10589,NLBZM,NLVER,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10590,NLBZM,NLSMB,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10591,NLBZM,NLSHH,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
type_list = list(df_main.columns[3:])

In [15]:
for i in type_list:
    df_chance[i] = df_chance[i]/df_chance.trip_count

In [68]:
pickle.dump( df_chance, open( "data/probability_df.p", "wb" ) )

In [60]:
test_100 = df_chance.head(100)

In [61]:
test_100

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,0.208790,0.353012,0.132354,0.010258,0.0,0.042945,0.000000,...,0.000503,0.000704,0.001106,0.000503,0.003922,0.000402,0.0,0.000000,0.000000,0.0
1,NLRTM,NLMOE,2686,0.026061,0.589724,0.053611,0.000745,0.0,0.020849,0.000000,...,0.000745,0.000000,0.000372,0.000000,0.000372,0.000000,0.0,0.000000,0.000000,0.0
2,NLRTM,NLVLI,2153,0.088249,0.305620,0.192290,0.004645,0.0,0.065026,0.000000,...,0.000000,0.000929,0.000929,0.000464,0.000000,0.000000,0.0,0.000464,0.000000,0.0
3,NLAMS,NLAER,2119,0.002832,0.062294,0.000000,0.000944,0.0,0.003303,0.000000,...,0.000000,0.000472,0.002360,0.017461,0.000000,0.001416,0.0,0.000472,0.000472,0.0
4,NLRTM,NLRTM,1794,0.202341,0.398551,0.051282,0.008361,0.0,0.091416,0.000557,...,0.002230,0.000000,0.000000,0.000000,0.001115,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NLNIJ,NLAMS,352,0.011364,0.411932,0.028409,0.065341,0.0,0.000000,0.000000,...,0.000000,0.011364,0.000000,0.000000,0.002841,0.000000,0.0,0.000000,0.002841,0.0
96,NLFAR,NLRTM,342,0.000000,0.903509,0.029240,0.005848,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.0,0.000000,0.000000,0.0
97,NLRTM,NLVSN,342,0.000000,0.602339,0.368421,0.000000,0.0,0.011696,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
98,NLPER,NLAMS,328,0.027439,0.387195,0.097561,0.000000,0.0,0.018293,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [62]:
def random_vessel_generator(df_prob):
    # create dict to store random prob based values later on
    main_dict = {i:[] for i in df_prob.columns}

    # loop over all rows of the probability dataframe
    for i in range(len(df_prob)):
        # copy origin, destination and count from original df
        for x in df_prob.columns[:3]:
            main_dict[x].append(df_prob.iloc[i,:][x])

        # find probability, items to pick, and the number of vessels to generate in total
        prob = list(df_prob.iloc[i, 3:].values)
        to_pick = list(df_prob.columns)
        to_pick = to_pick[3:]
        count = df_prob['trip_count'][i]

        # generate random vessels
        rand_vessels = np.random.choice(a=to_pick, size=count, replace=True, p=prob)
        unique, counts = np.unique(rand_vessels, return_counts=True)
        temp_dict = dict(zip(unique,counts))

        # append amount of random generated vessels right dict list
        for key in list(main_dict.keys())[3:]:
            if key in temp_dict.keys():
                main_dict[key].append(temp_dict[key])
            else:
                main_dict[key].append(0)

        # now make dict
        df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [65]:
import time
%time vs = random_vessel_generator(test_100)

CPU times: total: 312 ms
Wall time: 305 ms


In [66]:
vs

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2115,3439,1313,99,0,441,0,...,3,13,8,4,40,5,0,0,0,0
1,NLRTM,NLMOE,2686,58,1548,153,2,0,50,0,...,3,0,0,0,0,0,0,0,0,0
2,NLRTM,NLVLI,2153,156,692,404,14,0,157,0,...,0,3,0,1,0,0,0,0,0,0
3,NLAMS,NLAER,2119,7,119,0,0,0,2,0,...,0,1,7,39,0,1,0,1,1,0
4,NLRTM,NLRTM,1794,350,746,113,14,0,157,2,...,9,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NLNIJ,NLAMS,352,3,150,8,18,0,0,0,...,0,7,0,0,1,0,0,0,1,0
96,NLFAR,NLRTM,342,0,314,9,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
97,NLRTM,NLVSN,342,0,197,134,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
98,NLPER,NLAMS,328,6,125,28,0,0,9,0,...,0,0,0,0,0,0,0,0,0,0


## Multiply with factors and calculate fuel consumption

In [ ]:
ship_data = pd.read_excel('data/ship_types.xlsx')
ship_data.fillna(0, inplace=True)

In [ ]:
ship_data.loc[:,['RWS-class', 'Factor']]

In [ ]:
df_main

In [ ]:
ship_data = dict(zip(ship_data['RWS-class'],ship_data['Factor']))

In [ ]:
def flow_computation(df):
    flows = {}
    for i in range(len(df)-1):
        a = df.iloc[:,3:]
        flow = 0
        for row in a.columns:
            flow += ship_data[row] * a[row][i]

        flows[(df.origin[i],df.destination[i])]= (flow/365)
    return flows

In [ ]:
main10 = df_main.head(10)

In [ ]:
flows = flow_computation(main10)

In [ ]:
flows

In [ ]:
flows = dict(sorted(flows.items(), key=lambda item: item[1], reverse=True))

In [ ]:
flows

## Get and clean harbour data NL, Ger and Bel

In [ ]:
## Now get coordinates harbours
pd.read_csv('data/harbour_codes_coords/2021-2 UNLOCODE CodeListPart1.csv', encoding="ISO-8859-1", header=None, sep=",",
            index_col=None)

In [ ]:
#first identify all files in 2021 folder and save their names
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('data/harbour_codes_coords'):
    f.extend(filenames)
    break

In [ ]:
d_type_dict = {}
for i in range(12):
    d_type_dict[i] = str

In [ ]:

#empty list to store data
li = []

#read everything in as object for now
for filename in f:
    df = pd.read_csv("data/harbour_codes_coords/" + str(filename), sep=',', index_col=None, header=None,
                     encoding="ISO-8859-1", dtype = d_type_dict)
    print(filename, 'handled')
    li.append(df)

In [ ]:
#create df
df_harbour_codes = pd.concat(li, axis=0, ignore_index=True)
df_benelux = df_harbour_codes.loc[(df_harbour_codes[1] == 'NL')]
# df_benelux = df_harbour_codes.loc[
#     (df_harbour_codes[1] == 'NL') | (df_harbour_codes[1] == 'DE') | (df_harbour_codes[1] == 'BE') | (
#                 df_harbour_codes[1] == 'LU')]

In [ ]:
df_benelux.fillna(np.nan, inplace=True)

In [ ]:
# fix layout and column names
# now drop empty columns
df_benelux.rename(columns={0: 'mutation', 1: 'country', 2: 'city_abbr', 3: 'city_full', 4: 'city_full_2', 5: 'subdivision', 6: 'function', 7:'status', 8:'date', 9:'iata_code', 10: 'coords', 11: 'comments'}, inplace=True)

In [ ]:
df_benelux

In [ ]:
df_benelux = df_benelux.loc[:,['country','city_abbr','function', 'coords']]

In [ ]:
df_benelux.reset_index(inplace=True,drop=True)

In [ ]:
df_benelux.to_csv('data/cleaned_harbours.csv')

In [ ]:
d_type_dict = {}
for i in range(4):
    d_type_dict[i] = str

In [ ]:
df_benelux = pd.read_csv('data/cleaned_harbours.csv', index_col=0, dtype=d_type_dict)

In [ ]:
df_benelux.dropna(subset=['country', 'city_abbr', 'coords'], inplace = True)

In [ ]:
for i in df_benelux.columns:
    df_benelux[i] = df_benelux[i].astype('|S80')
    df_benelux[i] = df_benelux[i].apply(lambda x: x.decode('utf-8'))

In [ ]:
df_benelux.replace('nan', np.nan)

In [ ]:
df_benelux['harbour_code'] = df_benelux.country+df_benelux.city_abbr

In [ ]:
df_benelux